# Use LangChain with Mistral in Azure AI and Azure ML

You can use Mistral models deployed in Azure AI and Azure ML with `langchain` to create more sophisticated intelligent applications. Use `langchain_mistralai` package and configure it as follows:

- `endpoint`: Use the endpoint URL from your deployment. Do not include either `v1/chat/completions` as this is included automatically by the client.
- `api_key`: Use your API key.

Notice that Mistral supports only chat completions API.

## Prerequisites

Install `langchain-core` and `langchain_mistralai`:

```bash
pip install langchain-core langchain_mistralai
```

## Example

The following example demonstrate how to create a chain that uses a Mistral model deployed in Azure AI and Azure ML. The chain has been configured with a `ConversationBufferMemory`. This example has been adapted from [LangChain official documentation](https://python.langchain.com/docs/modules/memory/adding_memory).

In [ ]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
from langchain_mistralai.chat_models import ChatMistralAI

In the below prompt, we have two input keys: one for the actual input (`human_input`), and another for the input from the `Memory` class (`chat_history`).

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human. You love making references to french culture on your answers."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{human_input}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

Let's create an instance of our Mistral model deployed in Azure AI or Azure ML:

In [ ]:
chat_model = ChatMistralAI(
    endpoint="https://<endpoint>.<region>.inference.ai.azure.com",
    mistral_api_key="<key>",
)

> Tip: Alternatively, you can configure your API key in the environment variables `MISTRAL_API_KEY`.

We create the chain as follows:

In [ ]:
chat_llm_chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    memory=memory,
    verbose=True,
)

We can see how it works:

In [ ]:
chat_llm_chain.predict(human_input="Hi there my friend")

In [ ]:
chat_llm_chain.predict(
    human_input="I'm thinking on a present for my mother. Any advise?"
)